In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent/HOTS


In [2]:
import os
import pickle
import datetime
from mix_Network import *

dataset = 'nmnist'
records_path = '../Records'
timestr = datetime.datetime.now().date().isoformat()
timestr = '2021-02-13'
verbose = True

%mkdir -p ../Records
%mkdir -p ../Records/EXP_03_NMNIST

homeo = True
sigma = None
pooling = False
homeinv = False
jitter = False
tau = 5
krnlinit = 'first'
nblay = 3
nbclust = 4

ds = 3/2
ds = 750
nb_train = int(7500//ds)
nb_test = int(2500//ds)

In [3]:
import time
def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')

In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader
torch.set_default_tensor_type("torch.DoubleTensor") # -> torch.tensor([1.2, 3]).dtype = torch.float64
criterion = torch.nn.NLLLoss(reduction="mean") # loss divided by output size
# https://sebastianraschka.com/faq/docs/pytorch-crossentropy.html
class LogisticRegressionModel(torch.nn.Module):
    #torch.nn.Module -> Base class for all neural network modules
    def __init__(self, N, n_classes, bias=True):
        super(LogisticRegressionModel, self).__init__() 
        self.linear = torch.nn.Linear(N, n_classes, bias=bias)
        self.nl = torch.nn.LogSoftmax(dim=1)

    def forward(self, factors):
        return self.nl(self.linear(factors))

In [5]:
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 9 + 1
batch_size = 256
n_classes=10
amsgrad = False # gives similar results
amsgrad = True  # gives similar results

def fit_raw_data(dataset, 
            nb_digit,
            learning_rate=learning_rate,
            batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=False, #**kwargs
        ):
    
    learningset = tonic.datasets.NMNIST(save_to='../Data/',
                                        train=True,
                                        transform=tonic.transforms.AERtoVector())
    loader = tonic.datasets.DataLoader(learningset, shuffle=True)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'device -> {device}')

    N = 34*34*2
    n_classes = 10
    logistic_model = LogisticRegressionModel(N, n_classes)
    logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(
        logistic_model.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
    )
    
    for epoch in range(int(num_epochs)):
        losses = []
        for i in range(nb_digit):
            X_, labels_ = next(iter(loader))
            X_, labels_ = X_.to(device), labels_.to(device)
            outputs = logistic_model(X_)
            #target = torch.argmax(torch.squeeze(labels_,dim=1), 1)
            loss = criterion(outputs, torch.nn.functional.one_hot(labels_, num_classes=n_classes))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        if verbose and (epoch % (num_epochs // 32) == 0):
            print(f"Iteration: {epoch} - Loss: {np.mean(losses):.5f}")
            
    return logistic_model, losses

In [6]:
tic()
model, loss = fit_raw_data(dataset, 
            nb_train,
            learning_rate=learning_rate,
            batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=True,
        )
toc()

Using downloaded and verified file: ../Data/nmnist_train.zip
Extracting ../Data/nmnist_train.zip to ../Data/
device -> cpu
Iteration: 0 - Loss: 5.30066
Iteration: 16 - Loss: 2.13704
Iteration: 32 - Loss: 1.66638
Iteration: 48 - Loss: 1.56062
Iteration: 64 - Loss: 1.51190
Iteration: 80 - Loss: 1.23469
Iteration: 96 - Loss: 1.16081
Iteration: 112 - Loss: 1.15595
Iteration: 128 - Loss: 1.13485
Iteration: 144 - Loss: 1.05592
Iteration: 160 - Loss: 1.03762
Iteration: 176 - Loss: 1.06497
Iteration: 192 - Loss: 0.98060
Iteration: 208 - Loss: 0.97420
Iteration: 224 - Loss: 0.87093
Iteration: 240 - Loss: 0.89340
Iteration: 256 - Loss: 0.85311
Iteration: 272 - Loss: 0.70053
Iteration: 288 - Loss: 0.75226
Iteration: 304 - Loss: 0.81549
Iteration: 320 - Loss: 0.78877
Iteration: 336 - Loss: 0.80841
Iteration: 352 - Loss: 0.77050
Iteration: 368 - Loss: 0.73178
Iteration: 384 - Loss: 0.69867
Iteration: 400 - Loss: 0.74289
Iteration: 416 - Loss: 0.64376
Iteration: 432 - Loss: 0.72699
Iteration: 448 - 

In [7]:
import os
import pickle
f_name = f'{records_path}/{dataset}/models/{timestr}_LRmodel_raw.pkl'
with open(f_name, 'wb') as file:
    pickle.dump([model], file, pickle.HIGHEST_PROTOCOL)

In [11]:
tonic.datasets.DataLoader?

Init signature:
tonic.datasets.DataLoader(
    dataset: torch.utils.data.dataset.Dataset[+T_co],
    batch_size: Optional[int] = 1,
    shuffle: bool = False,
    sampler: Optional[torch.utils.data.sampler.Sampler[int]] = None,
    batch_sampler: Optional[torch.utils.data.sampler.Sampler[Sequence[int]]] = None,
    num_workers: int = 0,
    collate_fn: Callable[[List[~T]], Any] = None,
    pin_memory: bool = False,
    drop_last: bool = False,
    timeout: float = 0,
    worker_init_fn: Callable[[int], NoneType] = None,
    multiprocessing_context=None,
    generator=None,
    *,
    prefetch_factor: int = 2,
    persistent_workers: bool = False,
)
Docstring:     
Data loader. Combines a dataset and a sampler, and provides an iterable over
the given dataset.

The :class:`~torch.utils.data.DataLoader` supports both map-style and
iterable-style datasets with single- or multi-process loading, customizing
loading order and optional automatic batching (collation) and memory pinning.

See :p

In [12]:
learningset = tonic.datasets.NMNIST(save_to='../Data/',
                                train=False,
                                transform=tonic.transforms.AERtoVector())

generator=torch.Generator().manual_seed(42)
sampler = torch.utils.data.RandomSampler(full_dataset, replacement=True, num_samples=nb_test, generator=generator)
loader = tonic.datasets.DataLoader(learningset, shuffle=True)


Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/


In [17]:
loader.sampler.num_samples


10000

In [9]:
def predict_data(model, # gamma=gamma,
            verbose=False, **kwargs
        ):
    
    learningset = tonic.datasets.NMNIST(save_to='../Data/',
                                    train=False,
                                    transform=tonic.transforms.AERtoVector())
    loader = tonic.datasets.DataLoader(learningset, shuffle=True)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    logistic_model = model.to(device)
    
    pred_target = []
    
    X, label = next(iter(loader))
    X = X.to(device)
    
    outputs = logistic_model(X)

    pred_target.append(torch.argmax(outputs, dim=1).item())

    return pred_target, outputs

In [10]:
pred, output = predict_data(model)

Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/


NameError: name 'pred_target' is not defined

# Testing tonic functional aer_to_vect()

# Using batches

Pour le moment c'est trop long, l'idée était de regrouper différents digits, de faire un shuffle pour que le fit ne se fasse pas sur tout les events d'un digit, puis tous les events de l'autre etc... C'est l'idée du batch mais du coup ça fait des gros batch (environ 4000 x nb_digit au lieu de 256). Je sais pas encore comment faire de petits batchs avec différents digits tout en utilisant le dataloader.  

In [ ]:
f_name = 'lala.pkl'
f_name[:-4]